# Matrix and Vocabulary Construction

In [91]:
import pandas as pd

import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps

from collections import OrderedDict

FILE_PATH = '../data/estadao_noticias_eleicao.csv'

df = pd.read_csv(FILE_PATH, encoding = 'utf-8')
df = df.replace(np.NAN, "")

# criação de uma nova coluna para a junção do título da notícia com seu conteúdo
noticia = df.titulo + ' ' + df.subTitulo + ' ' + df.conteudo

**Obs:** O código abaixo que constrói a matriz de ocorrência foi copiado do repositório que está disponível em: https://github.com/allansales/information-retrieval/blob/master/Lab%203/coocurrence_matrix.ipynb

In [2]:
# O código abaixo está disponivel em: 
# https://github.com/allansales/information-retrieval/blob/master/Lab%203/coocurrence_matrix.ipynb

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

def  generate_tokens(noticia):
    tokens_lists = noticia.apply(lambda text: text.lower().split())
    tokens = [token for tokens_list in tokens_lists for token in tokens_list]
    return tokens

In [3]:
tokens = generate_tokens(noticia)
matrix, vocab = co_occurrence_matrix(tokens)
consultable_matrix = matrix.tocsr()

In [81]:
inv_vocab = {vocab[key]: key for key in vocab}

In [ ]:
# usar toarray() para gerar array, e zip para gerar duas listas e recuperar tanto o indice da palavra quanto 
# a frequencia. Depois ordenar pela frequencia e retornar as top-k palavras correspondentes.


print(consultable_matrix[vocab['dilma']])

In [99]:
def get_co_ocurrences(co_matrix, inv_vocab, term, N=3):
    # sparse to dense representation
    np_array = np.reshape(co_matrix[term].toarray(), -1)
    # get indice of N occurences
    return OrderedDict({inv_vocab[idx]: np_array[idx] for idx in (-np_array).argsort()[:N]})

In [101]:
get_co_ocurrences(consultable_matrix, inv_vocab, vocab['petrobrás'], N=3)

OrderedDict([('e', 216), ('paulo', 172), ('em', 77)])